# KdV Solitons
An important equation from the theory of solitons is the Korteweg-deVries (KdV) equation,
$$ \frac{\partial \rho}{\partial t} = -6\rho\frac{\partial \rho}{\partial x} - \frac{\partial^3 \rho}{\partial^3 x} $$
Write a program that solves the KdV equation using the explicit/implicit scheme
$$
\frac{\rho^{n+1}_j-\rho^{n}_j}{\Delta t} = -6D_j\,\rho^n_j-\frac{1}{2}

\Biggl(
\frac{\rho^{n}_{j+2}-2\rho^{n}_{j+1}+2\rho^{n}_{j-1}-\rho^{n}_{j-2}}{2\Delta x^3} + \frac{\rho^{n+1}_{j+2}-2\rho^{n+1}_{j+1}+2\rho^{n+1}_{j-1}-\rho^{n+1}_{j-2}}{2\Delta x^3}
\Biggr)
$$

where
$$
D_j = \frac{\rho^{n}_{j+1}-\rho^{n}_{j-1}}{2\Delta x}
$$

Use the Dirichlet boundary conditions $\rho(x = ±L/2) = 0$. Use the solitary wave solution of the KdV equation as an initial condition, $\rho(x, t) = 2\mbox{sech}^2(x − 4t)$. What do you observe?


In [34]:
import numpy as np

#parameters
tau=0.01
maxTime=10
tGridNum=int(np.ceil(maxTime/tau)) # we'd rather ceil than floor

L=10 #length
xGridSize=0.01
xGridNum=int(np.ceil(L/(xGridSize))) # integer

bufferSize=2 #set equal to the furthest away the derivative looks ahead on either side from the current point
#the discrete derivative we will use looks at most 2 grid locations away

#create index of X values for grid locations
N=np.linspace(start=0,stop=xGridNum-1,num=xGridNum)
xGrid=-L/2+(N+1/2)*xGridSize

#create index of time values
tIndices=np.linspace(start=0,stop=tGridNum-1,num=tGridNum) #largely defunct?
tGrid=np.linspace(start=0,stop=tGridNum-1,num=tGridNum)*tau

#create soliton
soliton=2/((np.cosh(xGrid))**2)

#create grid itself

grid=np.zeros((xGridNum+bufferSize,tGridNum))
#the extra (two) rows are out of bounds but can be indexed into from either side when taking the derivative for points at the edge of the simulated region

grid[:,0]=np.concatenate((soliton,np.zeros(bufferSize)))

#enforce boundary conditions
grid[0,0]=0
grid[-(1+bufferSize),0]=0#negative indexing allows me to loop back to the last element on that dimension, though I need to get past the buffer


Rearranging the implicit/explicit scheme yields:
$$
\rho^{n+1}_j =\rho^{n}_j -3\frac{\Delta t\,}{\Delta x}(\rho^{n}_{j+1}-\rho^{n}_{j-1})\,\rho^n_j
-\frac{\Delta t}{4\Delta x^3}
\bigl(
(\rho^{n}_{j+2}-2\rho^{n}_{j+1}+2\rho^{n}_{j-1}-\rho^{n}_{j-2}) + (\rho^{n+1}_{j+2}-2\rho^{n+1}_{j+1}+2\rho^{n+1}_{j-1}-\rho^{n+1}_{j-2})
\bigr)
$$
We can then fully rearrange so that $\rho^{n+1}$ terms are on the left, and $\rho^{n}$ terms are on the right

$$
\rho^{n+1}_j +\frac{\Delta t}{4\Delta x^3}(\rho^{n+1}_{j+2}-2\rho^{n+1}_{j+1}+2\rho^{n+1}_{j-1}-\rho^{n+1}_{j-2}) =\rho^{n}_j -3\frac{\Delta t\,}{\Delta x}(\rho^{n}_{j+1}-\rho^{n}_{j-1})\,\rho^n_j
-\frac{\Delta t}{4\Delta x^3}
\bigl(
(\rho^{n}_{j+2}-2\rho^{n}_{j+1}+2\rho^{n}_{j-1}-\rho^{n}_{j-2})
\bigr)
$$

We can clean up this expression by writing a vectorized form for all $\rho_j$ at a given timestep $n$.
We write $\rho^{n}$ to mean a column vector of $\rho^n_j$ with the j index disappearing as we are working with all rows at once.

Then we can write operations such as $$(\rho^{n}_{j+2}-2\rho^{n}_{j+1}+2\rho^{n}_{j-1}-\rho^{n}_{j-2})$$ as matrix products on $\rho^n$.
We start with the left hand side, defining
$$\alpha=\frac{\Delta t}{4\Delta x^3}$\mbox{ and } A\rho^{n}_j=(\rho^{n}_{j+2}-2\rho^{n}_{j}+2\rho^{n}_{j-1}-\rho^{n}_{j-2})$$
Where $n$ and $n+1$ are equally valid timesteps on which to apply $ A $ .

We get
$$
\rho^{n+1}_j +\alpha A\rho^{n+1} =\rho^{n}_j -\alpha A \rho^{n}

-3\frac{\Delta t\,}{\Delta x}(\rho^{n}_{j+1}-\rho^{n}_{j-1})\,\rho^n_j

$$

In the interest of bringing this completely, or close to completely in matrix terms, we can notice that $\rho^n_j$ should return itself for each j in the matrix version of the equation. That means we replace$\rho^n_j$ with $I\rho^n$.
Additionally, we have another matrix and prefactor to define.
We define
$$ \beta=3\frac{\Delta t\,}{\Delta x}\mbox{ and } B\rho^n_j = (\rho^{n}_{j+1}-\rho^{n}_{j-1})$$

Simplifying fully, we get

$$
I\rho^{n+1} +\alpha A\rho^{n+1} =I\rho^{n} -\alpha A \rho^{n}-3\beta (B\rho^n)\odot\rho^n
$$
Where $\odot$ means the hadamard (element-wise) product rather than the matrix product present in the rest of the equation.

Still, this can be further made matrix-like by noticing
$$
(B\rho^n)\odot\rho^n = \mbox{diag}(\rho^n)B\rho^n
$$

giving
$$
(I +\alpha A)\rho^{n+1} =
\bigl(
I -\alpha A -3\beta \mbox{diag}(\rho^n)B
\bigr)\rho^n
$$

In [28]:

#begin the loop

for j in range(1,tGridNum-1):
    for i in range(1,xGridNum-1):



SyntaxError: expected ':' (1629645956.py, line 4)